In [1]:
import torch
from torch.optim import Adam
from tqdm import tqdm
import numpy as np

In [2]:
%run dataprep.ipynb

In [3]:
%run modell.ipynb

In [4]:
DEVICE = 'cuda'

In [5]:
dataloaders = read_data(TRAIN_PATH, TEST_PATH)

In [6]:
bert_for_classification = BertForSequenceClassification(num_classes=6).to(DEVICE)
for p in bert_for_classification.model.parameters():
    p.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def train_model(train_dl, model, optimizer, criterion, optional_metric, device='cuda'):
    
    # define the optimization
    loss_agg = 0
    optional_metric_agg = 0
        
    for iteration, inputs in tqdm(enumerate(train_dl)):
        # clear the gradients
        inputs = {k: v.to(device) for (k, v) in inputs.items()}
        # targets = targets.to(device)
        optimizer.zero_grad()
        # compute the model output
        logits = model(inputs)
        # calculate loss
        loss = criterion(logits, inputs['targets'])
        # credit assignment
        loss.backward()
        # update model weights
        optimizer.step()
        
        #BCELoss or MSELoss
        loss_agg += loss.data.item()
        # optional_metric_agg += optional_metric().to(device)(torch.max(probs, 1)[1],targets).item()

    loss_agg /= len(train_dl)
    # optional_metric_agg /= len(train_dl)
    return (loss_agg, )#optional_metric_agg)#optional_metric_agg

In [8]:
def test_model(test_dl, model, criterion, optional_metric, device = 'cuda'):
    
    loss_agg = 0
    optional_metric_agg = 0
    
    model.eval()
    with torch.no_grad():
        for i, inputs in tqdm(enumerate(test_dl)):
            inputs = {k: v.to(device) for (k, v) in inputs.items()}
            # targets = targets.to(device)
            # evaluate the model on the test set
            logits = model(inputs)
            loss = criterion(logits, inputs['targets'])
            
            loss_agg += loss.data.item()
            # optional_metric_agg += optional_metric().to(device)(torch.max(probs, 1)[1], targets).item()
        
        loss_agg /= (len(test_dl))
        # optional_metric_agg /= (len(test_dl))
    return (loss_agg, ) #optional_metric_agg) # optional_metric_agg

In [18]:
def train_test_data(dataloaders, model, optimizer, criterion, optional_metric):
    
    metrics = []
    for epoch in range(3):
        
        train_result = train_model(dataloaders.train, model, optimizer, criterion, optional_metric)
        # test_result = test_model(dataloaders.test, model, criterion, optional_metric)
        
        metrics.append([*train_result])
    return np.array(metrics)

In [19]:
loss = nn.CrossEntropyLoss()
optimizer = Adam(bert_for_classification.parameters(), lr = 0.001)

In [20]:
metrics = train_test_data(dataloaders, bert_for_classification, optimizer, loss, None)

250it [01:01,  4.07it/s]
250it [01:02,  3.99it/s]
250it [01:03,  3.93it/s]
